<a href="https://colab.research.google.com/github/ardx160/ai-demo-people-zone-counter/blob/main/%5BDemo%5D_Zone_People_Detection_and_Count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello and welcome..

..to this awesome Google Colab workshop, where you will learn how to do some cool things with code and AI. Before we start, here are some things that you need to prepare:

### 1. You need to have a Google account to run the colab. If you don’t have one, you can create one for free here.

### 2. You need to open this bit.ly link: `bit.ly/ahmdemoai`. This will take you to the colab notebook that we will use for the workshop.

### 3. You can also bookmark this link for future reference. You need to make a copy of the colab notebook in your Google Drive. To do this, click on the File menu, then select Save a copy in Drive. This will allow you to edit and run the code in the notebook.

### 4. You need to have a good internet connection and a web browser that supports Google Colab. You can check the system requirements here.


That’s all you need to prepare for the workshop. If you have any questions or issues, please let me know. I’m here to help you along the way. I hope you are excited and ready to have some fun with code and AI. 😊



# A. Download and Import Libraries

*   Welcome to this Google Colab notebook, where you will learn how to do cool things with code. In this cell, you will download and import the libraries that you need. These libraries will help you with tasks like deep learning, computer vision, image processing, web applications, and more. You will also download and compress a sample video file. You will then see the video in the output.

*   To run the cell, you can click on the play button on the left side of the cell. You will see a spinning icon while the cell is running, and a check mark when it is done. You will also see the output of the cell below the code. If you see any errors or warnings in red color, you need to fix them.

*   This cell might take some time to run, so please be patient. You can use this time to relax, chat, or read the code comments. Don’t close this tab or refresh the page, or you will lose your progress. I’m here to help you along the way. 😊







In [ ]:
# @title #### `Download and Import Libraries: Click play button to run this cell`

# Import libraries
import torch
import os
import numpy as np
from IPython import display
from shapely.geometry import Polygon
from shapely import affinity
import cv2
import datetime
from base64 import b64encode
import plotly.graph_objects as go
import imageio
from PIL import Image
import gc
from pathlib import Path
import subprocess

# Define the path for the temp folder
temp_path = '/tmp'

# Define the initial polygon coordinates
polygon = np.array([
    [540,  985 ],
    [1620, 985 ],
    [1620, 2065],
    [540,  2065]
], np.int32)

# Clone the yolov5 repository to the temp folder
%cd {temp_path}
!git clone -q https://github.com/ultralytics/yolov5 # Add -q flag to suppress output

# Install the required packages
!pip install -q -r yolov5/requirements.txt # Add -q flag to suppress output
!pip install -q supervision==0.2.0 # Add -q flag to suppress output
!pip install -q dash # Add -q flag to suppress output and remove comment

# Import dash and supervision
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import supervision as sv
# print("supervision", sv.__version__) # Comment out this line to hide the supervision version

# Download the video file to the temp folder
!wget -q --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1vVrEVMxucHgqGd7vAa501ASojbeGPhIr' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1vVrEVMxucHgqGd7vAa501ASojbeGPhIr" -O market-square.mp4 && rm -rf /tmp/cookies.txt # Add -q flag to suppress output
MARKET_SQUARE_VIDEO_PATH = os.path.join(temp_path, 'market-square.mp4')

# Load the yolov5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5x6')

# Compress the video and save it to the compressed path
crf = 28
compressed_path = "/content/market-square-result-c.mp4"
# Use the check_call function to execute the ffmpeg command
subprocess.check_call(f"ffmpeg -i {MARKET_SQUARE_VIDEO_PATH} -vcodec libx264 -crf {crf} -vf scale=640:-2 {compressed_path}", shell=True)
# Collect the garbage to free up memory
gc.collect()

# Clear the output of the cell
display.clear_output()

# Show the video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display.HTML("""<video width=400 controls>
          <source src="%s" type="video/mp4">
       </video>""" % data_url)


# B. AI Detection Capability

*   In this cell, you will see how AI can detect people in a crowded scene from one frame of a video. You will use the yolov5 model to detect and draw boxes around all the people in the frame, and show some information about each person. You will then display the annotated frame in the output. To run the cell, click on the play button on the left side of the cell, or press `Shift + Enter` on your keyboard. If you see any errors or warnings in red color, you need to fix them. You will be amazed by how many people the AI can detect in one frame. You can zoom in and out of the frame, count the people, find yourself or your friends, or admire the scene and the AI. Have fun with it, and don’t worry, I’m here to help you. 😊





In [ ]:
# @title #### `AI Detection Capability: Click play button to run this cell`

# extract video frame
generator = sv.get_video_frames_generator(MARKET_SQUARE_VIDEO_PATH)
iterator = iter(generator)
frame = next(iterator)

# detect
results = model(frame, size=1280)
detections = sv.Detections.from_yolov5(results)

# annotate
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
frame = box_annotator.annotate(scene=frame, detections=detections)

%matplotlib inline
sv.show_frame_in_notebook(frame, (16, 16))

# C. Zone-People AI Tracking
*   In this cell, you will process the whole video file and see how AI can detect people in different frames. You will use the polygons and zones that you defined or created, and the same yolov5 model. You will also set a minimum detection threshold, and see how the AI can track the time that each polygon has people in it. You will then compress and display the video in the output.

*   To run the cell, you can use the same methods as before. You will see a spinning icon, a check mark, and the output. If you see any errors or warnings, you need to fix them. You will see how the AI can detect and track people in the video. You can also change the minimum detection threshold, and compare the original and the compressed video. You can also run this cell again after creating a new polygon in cell D, and see how it affects the results. Have fun with it, and don’t worry, I’m here to help you. 😊



In [ ]:
# @title #### `Zone-People AI Tracking: Click play button to run this cell`

save_path = "/content/market-square-result.mp4"

if Path(save_path).exists():
  os.remove(save_path)
if Path(compressed_path).exists():
  os.remove(compressed_path)

# Define a function to get the time text from the frame index and the frame rate
def get_time_text(frame_index, frame_rate):
  elapsed_time = frame_index / frame_rate
  # Change the format to minutes and seconds
  time_text = f"{int(elapsed_time // 60):02d}:{int(elapsed_time % 60):02d}"
  return time_text

font = cv2.FONT_HERSHEY_SIMPLEX
color = (255, 255, 255)
rect_width = 200
rect_height = 80
margin = 10

polygons = [polygon]

video_info = sv.VideoInfo.from_video_path(MARKET_SQUARE_VIDEO_PATH)

colors = sv.ColorPalette.default()
thickness = 4
text_scale = 2
zones = []
zone_annotators = []
box_annotators = []
for index, polygon in enumerate(polygons):
  zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)
  zones.append(zone)
  zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=colors.by_idx(index), thickness=thickness*2, text_thickness=thickness*2, text_scale=text_scale*2)
  zone_annotators.append(zone_annotator)
  box_annotator = sv.BoxAnnotator(color=colors.by_idx(index), thickness=thickness, text_thickness=thickness, text_scale=text_scale)
  box_annotators.append(box_annotator)

# Create a list of frame indices for each polygon
polygon_frame_indices = [0] * len(polygons)

# Define a function to process each frame of the video, using the yolov5 model and the supervision library
def process_frame(frame: np.ndarray, i) -> np.ndarray:
  results = model(frame, size=1280)
  detections = sv.Detections.from_yolov5(results)
  detections = detections[(detections.class_id == 0) & (detections.confidence > 0.5)]

  # Loop through the zones, zone annotators, and box annotators
  for index, (zone, zone_annotator, box_annotator) in enumerate(zip(zones, zone_annotators, box_annotators)):
    mask = zone.trigger(detections=detections)
    detections_filtered = detections[mask]
    frame = box_annotator.annotate(scene=frame, detections=detections_filtered, skip_label=True)
    frame = zone_annotator.annotate(scene=frame)

    time_text = get_time_text(polygon_frame_indices[index], video_info.fps)
    text_size, _ = cv2.getTextSize(time_text, font, text_scale, thickness)
    x = int(polygons[index][0][0])
    y = int(polygons[index][0][1])
    x1 = x - margin
    y1 = y - margin
    x2 = x + text_size[0] + margin
    y2 = y + text_size[1] + margin

    frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), -1)
    frame = cv2.putText(frame, time_text, (x, y + text_size[1]), font, text_scale, color, thickness, cv2.LINE_AA)
    detection_count = len(detections_filtered)
    if detection_count >= min_detection:
      polygon_frame_indices[index] = i
  return frame

min_detection = 9 #@param {type:"integer"}

sv.process_video(source_path=MARKET_SQUARE_VIDEO_PATH, target_path=save_path, callback=process_frame)

crf = 28
os.system(f"ffmpeg -i {save_path} -vcodec libx264 -crf {crf} -vf scale=640:-2 {compressed_path}")

gc.collect()

mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display.HTML("""<video width=400 controls>
          <source src="%s" type="video/mp4">
       </video>""" % data_url)



# D. Zone Editor App
*   In this cell, you will create your own polygon zone on the video frame. You will use a dash app to draw a new rectangle on the figure and save it as a polygon. You will then see the coordinates of the new polygon in the output. You can use this polygon to define a new zone for the AI detection.

*   To run the cell, you only need to do it once. You will see a spinning icon, a check mark, and the output. If you see any errors or warnings, you need to fix them. You will see a dash app with a figure and a button. You can draw a new rectangle on the figure by clicking and dragging on it. You can also erase the existing rectangle by clicking on the eraser icon. You can then click on the button to save the new polygon and see its coordinates. You can repeat this process as many times as you want, without running the cell again. Have fun with it, and don’t worry, I’m here to help you. 😊

In [ ]:
# @title #### `Zone Editor App: Click play button to run this cell`


# Get a frame from the sample video and convert it to a PIL Image object
video = imageio.get_reader(MARKET_SQUARE_VIDEO_PATH, "ffmpeg")
frame = video.get_data(0)
frame = Image.fromarray(frame)

# Create a figure with the frame as background
fig = go.Figure()
fig.add_layout_image(
    dict(
        source=frame, # Pass the PIL Image object
        xref="x",
        yref="y",
        x=0,
        y=0, # Change the y position to 0
        sizex=video.get_meta_data()["size"][0], # Change the sizex to match the video width
        sizey=video.get_meta_data()["size"][1], # Change the sizey to match the video height
        sizing="stretch",
        layer="below"
    )
)

# Update the axes and layout
fig.update_xaxes(
    showgrid=False,
    zeroline=False,
    range=[0, video.get_meta_data()["size"][0]]
)
fig.update_yaxes(
    showgrid=False,
    zeroline=False,
    range=[video.get_meta_data()["size"][1], 0], # Reverse the y-axis range
    scaleanchor="x",
    scaleratio=1
)
fig.update_layout(
    title="Polygon Editor",
    dragmode="drawrect", # Allow drawing new rectangles
    hovermode=False,
    newshape=dict(line_color="cyan"),
    activeshape=dict(opacity=0.8) # Remove line_color property
)

# Create a dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H3("Draw a new rectangle on the figure and click the button to save it"),
    dcc.Graph(id="graph", figure=fig, config={'modeBarButtonsToAdd':['drawrect','eraseshape']}), # Add the config argument here
    html.Button("Save new polygon", id="button"), # Add a button to save the new polygon
    html.Div(id="output") # Add a div to display the output
])

# Define a dash callback
@app.callback(
    Output("output", "children"), # The output is the children of the output div
    Input("button", "n_clicks"), # The input is the number of clicks of the button
    State("graph", "relayoutData") # The state is the relayout data of the graph
)
def save_new_polygon(n_clicks, relayoutData):
    # Check if the button is clicked and the relayout data is not empty
    if n_clicks and relayoutData:
        # Get the x and y coordinates of the new rectangle
        x0 = relayoutData["shapes[0].x0"]
        y0 = relayoutData["shapes[0].y0"]
        x1 = relayoutData["shapes[0].x1"]
        y1 = relayoutData["shapes[0].y1"]
        # Create a new polygon array from the rectangle coordinates
        new_polygon = np.array([
            [x0, y0],
            [x1, y0],
            [x1, y1],
            [x0, y1]
        ], np.int32)

        global polygon
        polygon = new_polygon
        # Return the new polygon coordinates as the output
        return f"New polygon coordinates: {polygon}"
    # If the button is not clicked or the relayout data is empty, return an empty output
    else:
        return ""

# Run the app
app.run_server(mode="inline")
